In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
import re

/home/minhas/cgn-dp-24-1/culinary_compass/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset
df = pd.read_csv("/home/minhas/cgn-dp-24-1/culinary_compass/data/recipes.csv")

In [5]:
# non_veg_ingredients = ["chicken", "beef", "pork", "fish", "shrimp", "lamb", "eggs","egg"]
# df["DietaryCategory"] = df["RecipeIngredientParts"].apply(
#     lambda ingredients: "Non-Vegetarian" if any(item in str(ingredients).lower() for item in non_veg_ingredients) else "Vegetarian"
# )


In [3]:
df =df.iloc[0:3000]

In [4]:
# Define non-vegetarian ingredients
non_veg_ingredients = set([
    # Meat & Poultry
    "chicken", "beef", "pork", "mutton", "lamb", "turkey", "duck", "quail", "goat", "veal",
    "rabbit", "boar", "venison", "bison", "kangaroo", "goose", "pheasant", "pigeon", "elk",
    
    # Processed Meat Products
    "bacon", "ham", "sausage", "pepperoni", "salami", "chorizo", "pastrami", "prosciutto",
    "mortadella", "hot dog", "jerky", "liverwurst", "blood sausage", "scrapple",
    
    # Seafood
    "fish", "tuna", "salmon", "trout", "cod", "haddock", "mackerel", "sardine", "anchovy",
    "herring", "catfish", "bass", "snapper", "grouper", "halibut", "swordfish", "mahi mahi",
    "flounder", "eel", "shark", "sturgeon", "tilapia", "tuna steaks", "swordfish steaks",
    
    # Shellfish
    "shrimp", "prawns", "crab", "lobster", "crawfish", "squid", "octopus", "scallops",
    "mussels", "clams", "oysters", "abalone", "conch",
    
    # Animal-Based Ingredients
    "eggs", "gelatin", "lard", "suet", "tallow", "bone broth", "fish sauce", "oyster sauce",
    "shrimp paste", "anchovy paste", "worcestershire sauce", "caviar", "roe", "squid ink",
    
    # Organ Meats (Offal)
    "liver", "kidney", "heart", "brain", "tripe", "sweetbreads", "tongue", "gizzards"
])

# Function to classify recipes correctly
def classify_recipe(ingredients):
    # Step 1: Clean string formatting issues
    ingredients = str(ingredients).lower().replace('"', '').replace("c(", "").replace(")", "")

    # Step 2: Convert to a list of ingredients
    ingredient_list = [ing.strip() for ing in ingredients.split(",")]

    # Step 3: Check for partial matches
    if any(any(non_veg in ingredient for non_veg in non_veg_ingredients) for ingredient in ingredient_list):
        return "Non-Vegetarian"
    return "Vegetarian"


# Apply classification
df["DietaryCategory"] = df["RecipeIngredientParts"].apply(classify_recipe)

In [5]:
df['DietaryCategory'].unique()

array(['Vegetarian', 'Non-Vegetarian'], dtype=object)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.set_option('display.max_colwidth', None)

In [7]:
# df[["RecipeIngredientParts", "DietaryCategory"]]

In [8]:
# Initialize Sentence Transformer Model

model = SentenceTransformer("all-MiniLM-L6-v2")

In [9]:
# Step 3: Compute embeddings for ingredients
df["IngredientEmbedding"] = df["RecipeIngredientParts"].apply(lambda x: model.encode(str(x), convert_to_tensor=True).tolist())

In [10]:

# Step 4: Save embeddings to a new file
df.to_csv("/home/minhas/cgn-dp-24-1/culinary_compass/data/recipes_with_embeddings.csv", index=False)

In [11]:
# Step 5: Load dataset with precomputed embeddings
df = pd.read_csv("/home/minhas/cgn-dp-24-1/culinary_compass/data/recipes_with_embeddings.csv")

In [12]:
df["IngredientEmbedding"] = df["IngredientEmbedding"].apply(lambda x: torch.tensor(eval(x)))

In [13]:
def recommend_recipes(nutrients, ingredients, diet_preference):
    """Recommend recipes based on user nutrients, ingredients, and dietary preference."""

    # Filter dataset based on dietary preference
    if diet_preference != "Any":
        filtered_df = df[df["DietaryCategory"] == diet_preference].copy()
    else:
        filtered_df = df.copy()

    # Encode input ingredients
    input_ingredient_embedding = model.encode(" ".join(ingredients), convert_to_tensor=True)

    # Compute cosine similarity
    ingredient_similarities = util.pytorch_cos_sim(
        torch.stack(filtered_df["IngredientEmbedding"].tolist()), input_ingredient_embedding
    ).squeeze().numpy()

    # Normalize nutrient similarity
    nutrient_columns = ["Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", 
                        "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent"]
    
    df_nutrients = filtered_df[nutrient_columns].fillna(0)
    nutrient_array = df_nutrients.to_numpy()

    # Compute Euclidean distance
    input_nutrient_array = np.array([nutrients[col] for col in nutrient_columns]).reshape(1, -1)
    nutrient_distances = np.linalg.norm(nutrient_array - input_nutrient_array, axis=1)
    nutrient_similarities = 1 / (1 + nutrient_distances)

    # Compute final scores
    final_scores = (0.3 * nutrient_similarities) + (0.7 * ingredient_similarities)

    # Rank recipes and return top results
    filtered_df["SimilarityScore"] = final_scores
    top_recipes = filtered_df.sort_values(by="SimilarityScore", ascending=False).head(5)

    return top_recipes[["Name", "Images", "RecipeInstructions"]].to_dict(orient="records")


In [14]:
# Step 7: User input for testing
user_nutrients = {
    "Calories": 500, "FatContent": 20, "SaturatedFatContent": 5,
    "CholesterolContent": 10, "SodiumContent": 500, "CarbohydrateContent": 50,
    "FiberContent": 10, "SugarContent": 10, "ProteinContent": 30
}

user_ingredients = [ "Egg, whole, raw, frozen, salted, pasteurized", "Cheese, American, restaurant",
                    "Cheese, cotija, solid", "Crustaceans, crab, alaska king, raw",
                      "Mollusks, clam, mixed species, raw",  "Seaweed, wakame, raw",
                    "Cream cheese, full fat, block"]
# Define the diet preference
diet_preference = "Any"# You can set this to "Vegetarian", "Non-Vegetarian", or "Any" based on user input

# Step 8: Get recommendations
recommendations = recommend_recipes(user_nutrients, user_ingredients, diet_preference)

# Step 9: Display results
for recipe in recommendations:
    print(f"Name: {recipe['Name']}\nImage: {recipe['Images']}\nInstructions: {recipe['RecipeInstructions']}\n{'-'*50}")

Name: Shrimp and Gruyere Cheesecake
Image: character(0)
Instructions: c("cheese (about 5 oz.)", "1 Tsp.", "pepper   --GARNISHES IF DESIRED:--   Red pepper strips   Whole cooked shrimp   Basil leaves   --TOMATO SAUCE:--   1/4 cup chopped onion   1 Clove crushed garlic   1 Tbsp.", "olive oil   2 (14 Oz.) cans whole tomatoes  1 1/2 Tsp.", "dried Italian seasoning   1 bay leaf  Serve with tomato sauce, recipe to follow.", "Combine cracker crumbs and 1/4 cup  melted butter.", "Press into bottom of 9 inch springform pan.", "Chop cooked and  cleaned shrimp.", "Saute peppers, onion and garlic in 3 tablespoons melted butter  for 4-5 minutes until tender, add chopped shrimp and saute another minute.  Drain well and set aside. Beat cream cheese and mayonnaise at high speed until  light and fluffy, add eggs, one at a time, beating well after each addition.", 
"Gradually add milk beating on low speed just until blended.", "Stir in shrimp  mixture, shredded Gruyere or Swiss and 1 teaspoon pepper.", 

# End

In [ ]:
diet_preference = st.sidebar.selectbox("Dietary Preference:", ["Any", "Vegetarian", "Non-Vegetarian"])



non_veg_ingredients = ["chicken", "beef", "pork", "fish", "shrimp", "lamb", "eggs", "egg"]

df["DietaryCategory"] = df["RecipeIngredientParts"].apply(
    lambda ingredients: "Non-Vegetarian" if any(re.search(rf"\b{item}\b", str(ingredients).lower()) for item in non_veg_ingredients) else "Vegetarian"
)

# Display the updated DataFrame
print(df)


def recommend_recipes(nutrients, ingredients, diet_preference):
    """Recommend recipes based on user nutrients, ingredients, and dietary preference."""

    # Filter dataset based on dietary preference
    if diet_preference != "Any":
        filtered_df = df[df["DietaryCategory"] == diet_preference].copy()
    else:
        filtered_df = df.copy()

    # Encode input ingredients
    input_ingredient_embedding = model.encode(" ".join(ingredients), convert_to_tensor=True)

    # Compute cosine similarity
    ingredient_similarities = util.pytorch_cos_sim(
        torch.stack(filtered_df["IngredientEmbedding"].tolist()), input_ingredient_embedding
    ).squeeze().numpy()

    # Normalize nutrient similarity
    nutrient_columns = ["Calories", "FatContent", "SaturatedFatContent", "CholesterolContent", 
                        "SodiumContent", "CarbohydrateContent", "FiberContent", "SugarContent", "ProteinContent"]
    
    df_nutrients = filtered_df[nutrient_columns].fillna(0)
    nutrient_array = df_nutrients.to_numpy()

    # Compute Euclidean distance
    input_nutrient_array = np.array([nutrients[col] for col in nutrient_columns]).reshape(1, -1)
    nutrient_distances = np.linalg.norm(nutrient_array - input_nutrient_array, axis=1)
    nutrient_similarities = 1 / (1 + nutrient_distances)

    # Compute final scores
    final_scores = (0.6 * nutrient_similarities) + (0.4 * ingredient_similarities)

    # Rank recipes and return top results
    filtered_df["SimilarityScore"] = final_scores
    top_recipes = filtered_df.sort_values(by="SimilarityScore", ascending=False).head(5)

    return top_recipes[["Name", "Images", "RecipeInstructions"]].to_dict(orient="records")


recommendations = recommend_recipes(user_nutrients, user_ingredients, diet_preference)

